In [4]:
import warnings
import time
import json
from itertools import chain
from functools import partial
from typing import Dict, List

import requests
import pandas as pd
from fake_headers import Headers
from bs4 import BeautifulSoup
from returns.io import IO
from lxml import html

In [0]:
'''
Общий алгоритм работы
---------------------
-> Определь неизменяемые входные данные ({url, domain, headers{browser,os,headers}}).
-> Получить общее количество страниц пагинации с данными (товарами)
-> Получить url всех страниц пагинации (в случае e-katalog это порядковый номер страницы)
-> Получить список списков со всеми url конкретного товарова со всех страниц.
-> Распаковать все списки в один список.
-> Добавить ко всем url домен, как итог список окончательных ссылок к товару.
_____________________
'''

'\nОбщий алгоритм работы\n---------------------\n-> Определь неизменяемые входные данные ({url, domain, headers{browser,os,headers}}).\n-> Получить общее количество страниц пагинации с данными (товарами)\n-> Получить url всех страниц пагинации (в случае e-katalog это порядковый номер страницы)\n-> Получить список списков со всеми url конкретного товарова со всех страниц.\n-> Распаковать все списки в один список.\n-> Добавить ко всем url домен, как итог список окончательных ссылок к товару.\n_____________________\n'

In [0]:
warnings.filterwarnings('ignore')

In [0]:
class imdict(dict):
    def __hash__(self):
        return id(self)

    def _immutable(self, *args, **kws):
        raise TypeError('object is immutable')

    __setitem__ = _immutable
    __delitem__ = _immutable
    clear       = _immutable
    update      = _immutable
    setdefault  = _immutable
    pop         = _immutable
    popitem     = _immutable

In [0]:
data = imdict({'URL':'https://www.e-katalog.ru/list/61/',
               'DOMAIN': 'https://www.e-katalog.ru'})

headers =imdict({'browser':"chrome",
                 'os':"win",
                 'headers': True})

In [0]:
def generate_headers(headers_setting: Dict) -> Dict:
    '''
    Генерирует html заголовки запросов. В данном случае фейковые.
    '''
    #TODO Расширить количество аргументов.

    return Headers(
                   browser = headers_setting['browser'],
                   os = headers_setting['os'],
                   headers = headers_setting['headers']
                   ).generate()

In [0]:
def get_page_html(url: str, verify: bool = False, **kwargs) -> BeautifulSoup:
    '''На основе url получает всю html страницы, без потверждения ssl(опционально)'''

    time.sleep(1)
    return BeautifulSoup(requests.get(url, verify=verify, **kwargs).content, 'html.parser')

In [0]:
def get_product_amount_page(url: str, headers: Dict = generate_headers(headers)) -> IO[int]:
  
    '''
    Возвращает общее количество страниц для конкретного типа продукта
    | ---
    | url = базовый url страницы конкретного продукта (data['url'])
    | headers = Шапка html запроса в формате словаря.
    | ---
    '''

    def page_count(tree):
        return tree.xpath('//div[@class="ib page-num"]//a[last()]/text()')   

    return int(page_count(html.fromstring(requests.get(url, headers = headers).content))[0])

In [0]:
def collect_product_pages(url: str, pages_count: int) -> List[str]:
    '''
    *Получает список url страниц с товарами.*
    | ---
    | url = базовый url страницы конкретного продукта (data['url'])
    | pages_count = Общее количество страниц для продукта.
    | ---
    '''
    return list(map(lambda x: url + f"{x}/", range(1, pages_count+1)))

In [0]:
def collect_url_from_page(page_url: str, headers = generate_headers(headers), sleep: int = 1) -> IO[List[str]]:
    '''
    *Получает все url продуктов со страницы(получает список страницы), возвращает список url*
    | ---
    | page_url = url страницы со товармаи
    | headers = Шапка html запроса в формате словаря
    | sleep = Время задержки после запроса
    | ---
    '''

    def get_product_url(tree):
        return tree.xpath("//a[@class='model-short-title no-u no-u']/@href")
    
    time.sleep(sleep)
    return get_product_url(html.fromstring(requests.get(page_url, headers=headers).content))
    

In [0]:

def get_product_data(soup: BeautifulSoup) -> IO[Dict[str, str]]:
    '''Получает BeautifulSoup объект. Получает имя товара, и таблицу с характеристиками'''

    _ = dict()
    _.update({'Наименование товара' : soup.find("div", class_= "op1-tt").text})
    _.update({'Cost': soup.find('div', class_='desc-short-prices').find('a', class_='ib').text})

    for value in soup.find("table", class_="one-col").contents:

        try:
            _.update({value.find("span", class_="gloss").text : value.find("td", class_="val").text}) 

        except AttributeError:
            continue
        
    return _

In [0]:

def create_json(urls: List[str]) -> Dict[str, List[Dict[str, str]]]:

    global get_product_data, get_page_html

    def data_list(url):
        data = get_product_data(get_page_html(url))
        data.update({'url' : url})
        return data

    return {'SSD': list(map(data_list,urls))}

#json.dump(ssd, f, indent=4, ensure_ascii=False)

In [0]:
pages_urls = collect_product_pages(
                                   data['URL'], get_product_amount_page(
                                                                        data['URL'],
                                                                        generate_headers(headers)
                                                                        )
                                   )

In [0]:
all_url_product = list(
                       chain(
                             *list(
                                   map(
                                       collect_url_from_page, pages_urls
                                       )
                                   )
                             )
                       ) # Распоковка всех списков

In [0]:
full_url_path_to_product = list(
                                map(
                                    lambda x: data['DOMAIN'] + x, all_url_product
                                    )
                                )

In [0]:
data_ssd = create_json(full_url_path_to_product)

AttributeError: AttributeError: 'NoneType' object has no attribute 'text'

In [0]:
with open('ssd_data.json', mode='a', encoding='UTF8') as f:
    json.dump(data_ssd, f, indent=4, ensure_ascii=False)
    f.close()

NameError: NameError: name 'data_ssd' is not defined

In [53]:

# def get_product_price(soup: BeautifulSoup) -> IO[Dict[str, str]]:
#     '''Получает BeautifulSoup объект. Получает имя товара, и таблицу с характеристиками'''
#     global get_page_html

#     _ = dict()
#     _.update({'Наименование товара' : soup.find("div", class_= "op1-tt").text})
    
#     more_price = get_page_html(soup.find_all('div',class_='list-more-div-small blue-button h')[0].get('jsource'))
#     market_list_full = soup.find_all('a',class_='it-shop') + more_price.find_all('a', class_= r"\'it-shop\'")
#     price_product_list_full = soup.find('table',class_='where-buy-table').find_all('a') + \
#     more_price.find('table',class_=r"\'where-buy-table").find_all('a',class_ = not 'yel-but-2')

#     for key, value in zip(market_list_full,price_product_list_full):

#         try:
#             _.update({key.text: value.text}) 

#         except AttributeError:
#             continue

#     # else:
#     #     print(f'Нужная таблица не найдена или пуста.')

#     return _